# Style-Bert-VITS2 (ver 2.7.0) のGoogle Colabでの学習

Google Colab上でStyle-Bert-VITS2の学習を行うことができます。

このnotebookでは、通常使用ではあなたのGoogle Driveにフォルダ`Style-Bert-VITS2`を作り、その内部での作業を行います。他のフォルダには触れません。
Google Driveを使わない場合は、初期設定のところで適切なパスを指定してください。

## 流れ

### 学習を最初からやりたいとき
上から順に実行していけばいいです。音声合成に必要なファイルはGoogle Driveの`Style-Bert-VITS2/model_assets/`に保存されます。また、途中経過も`Style-Bert-VITS2/Data/`に保存されるので、学習を中断したり、途中から再開することもできます。

### 学習を途中から再開したいとき
0と1を行い、3の前処理は飛ばして、4から始めてください。スタイル分け5は、学習が終わったら必要なら行ってください。


## 0. 環境構築

Style-Bert-VITS2の環境をcolab上に構築します。ランタイムがT4等のGPUバックエンドになっていることを確認し、実行してください。

**注意**: このセルを実行した後に「セッションがクラッシュしました」「不明な理由により、セッションがクラッシュしました。」等の警告が出ますが、**無視してそのまま先へ**進んでください。（一度ランタイムを再起動させてnumpy<2を強制させるため `exit()` を呼んでいることからの措置です。）

In [1]:
import os

os.environ["PATH"] += ":/root/.cargo/bin"

!curl -LsSf https://astral.sh/uv/install.sh | sh
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!uv pip install --system -r requirements-colab.txt --no-progress
!python initialize.py --skip_default_models

exit()

downloading uv 0.9.10 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
fatal: destination path 'Style-Bert-VITS2' already exists and is not an empty directory.
/content/Style-Bert-VITS2
Using Python 3.12.12 environment at: /usr
Audited 26 packages in 230ms


In [1]:
# Google driveを使う方はこちらを実行してください。

from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


## 1. 初期設定

学習とその結果を保存するディレクトリ名を指定します。
Google driveの場合はそのまま実行、カスタマイズしたい方は変更して実行してください。

In [2]:
# 作業ディレクトリを移動
%cd /content/Style-Bert-VITS2/

# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = "/content/drive/MyDrive/Style-Bert-VITS2/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = "/content/drive/MyDrive/Style-Bert-VITS2/model_assets"

import yaml


with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

/content/Style-Bert-VITS2


## 2. 学習に使うデータ準備

すでに音声ファイル（1ファイル2-12秒程度）とその書き起こしデータがある場合は2.2を、ない場合は2.1を実行してください。

### 2.1 音声ファイルからのデータセットの作成（ある人はスキップ可）

音声ファイル（1ファイル2-12秒程度）とその書き起こしのデータセットを持っていない方は、（日本語の）音声ファイルのみから以下の手順でデータセットを作成することができます。Google drive上の`Style-Bert-VITS2/inputs/`フォルダに音声ファイル（wavやmp3等の通常の音声ファイル形式、1ファイルでも複数ファイルでも可）を置いて、下を実行すると、データセットが作られ、自動的に正しい場所へ配置されます。

**2024-06-02のVer 2.5以降**、`inputs/`フォルダにサブフォルダを2個以上作ってそこへ音声ファイルをスタイルに応じて振り分けて置くと、学習の際にサブディレクトリに応じたスタイルが自動的に作成されます。デフォルトスタイルのみでよい場合や手動でスタイルを後で作成する場合は`inputs/`直下へ入れれば大丈夫です。

In [20]:
# 0. 足りない部品をインストール（これを追加しました！）
!pip install faster-whisper

# 元となる音声ファイル（wav形式）を入れるディレクトリ
input_dir = "/content/Style-Bert-VITS2/inputs"
# モデル名（話者名）を入力
model_name = "AIchan-2025Autumn-v1"

# こういうふうに書き起こして欲しいという例文（句読点の入れ方・笑い方や固有名詞等）
initial_prompt = "こんにちは。元気ですか？　ふふっ、私は…ちゃんと元気だよ！！"

!python slice.py -i "{input_dir}" --model_name "{model_name}"
!python transcribe.py --model_name "{model_name}" --initial_prompt "{initial_prompt}" --language "ja"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 21.9 MB/s eta 0:00:00
11-20 14:25:36 |  INFO  | slice.py:167 | Found 2 audio files.
11-20 14:25:36 |WARNING | slice.py:169 | Output directory /content/drive/MyDrive/Style-Bert-VITS2/Data/AIchan-2025Autumn-v1/raw already exists, deleting...
Using cache found in /root/.cache/torch/hub/litagin02_silero-vad_master
  0%|          | 0/2 [00:00<?, ?it/s]Using cache found in /root/.cache/torch/hub/litagin02_silero-vad_master
/root/.cache/torch/hub/litagin02_silero-vad_master/utils_vad.py:127: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Pl

成功したらそのまま3へ進んでください

### 2.2 音声ファイルと書き起こしデータがすでにある場合

指示に従って適切にデータセットを配置してください。

次のセルを実行して、学習データをいれるフォルダ（1で設定した`dataset_root`）を作成します。

In [ ]:
import os

os.makedirs(dataset_root, exist_ok=True)

まず音声データと、書き起こしテキストを用意してください。

それを次のように配置します。
```
├── Data/
│   ├── {モデルの名前}
│   │   ├── esd.list
│   │   ├── raw/
│   │   │   ├── foo.wav
│   │   │   ├── bar.mp3
│   │   │   ├── style1/
│   │   │   │   ├── baz.wav
│   │   │   │   ├── qux.wav
│   │   │   ├── style2/
│   │   │   │   ├── corge.wav
│   │   │   │   ├── grault.wav
...
```

### 配置の仕方
- 上のように配置すると、`style1/`と`style2/`フォルダの内部（直下以外も含む）に入っている音声ファイルたちから、自動的にデフォルトスタイルに加えて`style1`と`style2`というスタイルが作成されます
- 特にスタイルを作る必要がない場合や、スタイル分類機能等でスタイルを作る場合は、`raw/`フォルダ直下に全てを配置してください。このように`raw/`のサブディレクトリの個数が0または1の場合は、スタイルはデフォルトスタイルのみが作成されます。
- 音声ファイルのフォーマットはwav形式以外にもmp3等の多くの音声ファイルに対応しています

### 書き起こしファイル`esd.list`

`Data/{モデルの名前}/esd.list` ファイルには、以下のフォーマットで各音声ファイルの情報を記述してください。


```
path/to/audio.wav(wavファイル以外でもこう書く)|{話者名}|{言語ID、ZHかJPかEN}|{書き起こしテキスト}
```

- ここで、最初の`path/to/audio.wav`は、`raw/`からの相対パスです。つまり、`raw/foo.wav`の場合は`foo.wav`、`raw/style1/bar.wav`の場合は`style1/bar.wav`となります。
- 拡張子がwavでない場合でも、`esd.list`には`wav`と書いてください、つまり、`raw/bar.mp3`の場合でも`bar.wav`と書いてください。


例：
```
foo.wav|hanako|JP|こんにちは、元気ですか？
bar.wav|taro|JP|はい、聞こえています……。何か用ですか？
style1/baz.wav|hanako|JP|今日はいい天気ですね。
style1/qux.wav|taro|JP|はい、そうですね。
...
english_teacher.wav|Mary|EN|How are you? I'm fine, thank you, and you?
...
```
もちろん日本語話者の単一話者データセットでも構いません。

## 3. 学習の前処理

次に学習の前処理を行います。必要なパラメータをここで指定します。次のセルに設定等を入力して実行してください。「～～かどうか」は`True`もしくは`False`を指定してください。

In [21]:
# 上でつけたフォルダの名前`Data/{model_name}/`
model_name = "AIchan-2025Autumn-v1"

# JP-Extra （日本語特化版）を使うかどうか。日本語の能力が向上する代わりに英語と中国語は使えなくなります。
use_jp_extra = True

# 学習のバッチサイズ。VRAMのはみ出具合に応じて調整してください。
batch_size = 4

# 学習のエポック数（データセットを合計何周するか）。
# 100で多すぎるほどかもしれませんが、もっと多くやると質が上がるのかもしれません。
epochs = 100

# 保存頻度。何ステップごとにモデルを保存するか。分からなければデフォルトのままで。
save_every_steps = 1000

# 音声ファイルの音量を正規化するかどうか
normalize = True

# 音声ファイルの開始・終了にある無音区間を削除するかどうか
trim = True

# 読みのエラーが出た場合にどうするか。
# "raise"ならテキスト前処理が終わったら中断、"skip"なら読めない行は学習に使わない、"use"なら無理やり使う
yomi_error = "skip"

上のセルが実行されたら、次のセルを実行して学習の前処理を行います。

In [22]:
from gradio_tabs.train import preprocess_all
from style_bert_vits2.nlp.japanese import pyopenjtalk_worker


pyopenjtalk_worker.initialize_worker()

preprocess_all(
    model_name=model_name,
    batch_size=batch_size,
    epochs=epochs,
    save_every_steps=save_every_steps,
    num_processes=2,
    normalize=normalize,
    trim=trim,
    freeze_EN_bert=False,
    freeze_JP_bert=False,
    freeze_ZH_bert=False,
    freeze_style=False,
    freeze_decoder=False,
    use_jp_extra=use_jp_extra,
    val_per_lang=0,
    log_interval=200,
    yomi_error=yomi_error,
)

11-20 14:29:13 |  INFO  | train.py:72 | Step 1: start initialization...
model_name: AIchan-2025Autumn-v1, batch_size: 4, epochs: 100, save_every_steps: 1000, freeze_ZH_bert: False, freeze_JP_bert: False, freeze_EN_bert: False, freeze_style: False, freeze_decoder: False, use_jp_extra: True
11-20 14:29:13 |WARNING | train.py:103 | Step 1: /content/drive/MyDrive/Style-Bert-VITS2/Data/AIchan-2025Autumn-v1/models already exists, so copy it to backup to /content/drive/MyDrive/Style-Bert-VITS2/Data/AIchan-2025Autumn-v1/models_backup
11-20 14:29:23 |SUCCESS | train.py:132 | Step 1: initialization finished.
11-20 14:29:23 |  INFO  | train.py:137 | Step 2: start resampling...
11-20 14:29:23 |  INFO  | subprocess.py:23 | Running: resample.py -i /content/drive/MyDrive/Style-Bert-VITS2/Data/AIchan-2025Autumn-v1/raw -o /content/drive/MyDrive/Style-Bert-VITS2/Data/AIchan-2025Autumn-v1/wavs --num_processes 2 --sr 44100 --normalize --trim
11-20 14:29:41 |SUCCESS | subprocess.py:38 | Success: resample.p

(True, 'Success: 全ての前処理が完了しました。ターミナルを確認しておかしいところがないか確認するのをおすすめします。')

## 4. 学習

前処理が正常に終わったら、学習を行います。次のセルを実行すると学習が始まります。

学習の結果は、上で指定した`save_every_steps`の間隔で、Google Driveの中の`Style-Bert-VITS2/Data/{モデルの名前}/model_assets/`フォルダに保存されます。

このフォルダをダウンロードし、ローカルのStyle-Bert-VITS2の`model_assets`フォルダに上書きすれば、学習結果を使うことができます。

In [23]:
# 上でつけたモデル名を入力。学習を途中からする場合はきちんとモデルが保存されているフォルダ名を入力。
model_name = "AIchan-2025Autumn-v1"


import yaml
from gradio_tabs.train import get_path

paths = get_path(model_name)
dataset_path = str(paths.dataset_path)
config_path = str(paths.config_path)

with open("default_config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)
yml_data["model_name"] = model_name
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)

In [24]:
# 日本語特化版を「使う」場合
!python train_ms_jp_extra.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

2025-11-20 14:42:38.616174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763649758.648366   24175 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763649758.658854   24175 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763649758.682437   24175 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763649758.682470   24175 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763649758.682476   24175 computation_placer.cc:177] computation placer alr

In [ ]:
# 日本語特化版を「使わない」場合
!python train_ms.py --config {config_path} --model {dataset_path} --assets_root {assets_root}

In [ ]:
# 学習結果を試す・マージ・スタイル分けはこちらから
!python app.py --share

2025-11-20 15:00:12.758311: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763650812.797587   28522 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763650812.811342   28522 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763650812.850573   28522 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763650812.850611   28522 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763650812.850620   28522 computation_placer.cc:177] computation placer alr

In [ ]:
# ONNX変換は、変換したいsafetensorsファイルを指定してこのセルを実行してください。
!python convert_onnx.py --model "Data/your_model/your_model_e100_s10000.safetensors"